In [1]:
from sklearn.metrics import mean_absolute_error as MSE
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

# Loading our data

In [2]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")

price = df['NO2_price'].values.reshape(-1,1)
fload = df['NO2_load_forecasted'].values.reshape(-1,1)
fgen = df['NO2_generation_forecast'].values.reshape(-1,1)


price_days = seperate_column_to_days(price)
fload_days = seperate_column_to_days(fload)
fgen_days = seperate_column_to_days(fgen)

# Structuring our data

In [3]:
lookbehind = 7
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
stride = 24
hour_in_days = int(no_hours / stride)

price_dataset = []
fload_dataset = []
fgen_dataset = []

for i in range(len(price_days) - hour_in_days+1):
    price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))
    fload_dataset.append(np.concatenate((fload_days[i:i+hour_in_days])))
    fgen_dataset.append(np.concatenate((fgen_days[i:i+hour_in_days])))

price_dataset = np.array(price_dataset)
fload_dataset = np.array(fload_dataset)
fgen_dataset = np.array(fgen_dataset)

scaler = MinMaxScaler()
price_dataset = scaler.fit_transform(price_dataset[:,:,0])
price_dataset = price_dataset[..., np.newaxis].astype(np.float32)

fload_dataset = scaler.fit_transform(fload_dataset[:,:,0])
fload_dataset = fload_dataset[..., np.newaxis].astype(np.float32)

fgen_dataset = scaler.fit_transform(fgen_dataset[:,:,0])
fgen_dataset = fgen_dataset[..., np.newaxis].astype(np.float32)

dataset_forecast = np.concatenate((price_dataset, fload_dataset, fgen_dataset), axis=2)

n,m,k = dataset_forecast.shape

# Train test split

In [4]:
train = int(0.7*n)
valid = int(0.9*n)
x_train, X_train = price_dataset[:train, :input_width], dataset_forecast[:train, :input_width]
x_valid, X_valid = price_dataset[train:valid, :input_width], dataset_forecast[train:valid, :input_width]
x_test, X_test = price_dataset[valid:, :input_width], dataset_forecast[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset_forecast[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

# Setting up the models

## Common parameters

In [5]:
epochs = 2
loss = "mse" 
optimizer = "adam"

univ_neurons = 128

multi_neurons_first = 128
multi_neurons_second = 64
filters = 64
kernel_size = 3
strides = 2

# Grid search parameters

In [6]:
batch_size = [200, 400, 600, 800]
eta_list = np.logspace(-1, -4, 6)

# Setting up the univariate SimpleRNN

In [7]:
rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(univ_neurons, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(univ_neurons, return_sequences=True),
    keras.layers.Dense(horizon)
])

rnn.compile(loss=loss, optimizer=optimizer)
rnn.fit(x_train, Y_train, epochs=epochs, batch_size = 500, validation_data=(x_valid, Y_valid))
Y_pred_rnn = rnn.predict(x_test)

Epoch 1/2
4/4 [==============================] - 5s 689ms/step - loss: 0.1366 - val_loss: 0.0512
Epoch 2/2
4/4 [==============================] - 2s 515ms/step - loss: 0.0527 - val_loss: 0.0158


In [8]:
print(Y_test.shape)
print(Y_pred_rnn.shape)

print(MSE(Y_test[:,-1], Y_pred_rnn[:,-1]))

(215, 168, 24)
(215, 168, 24)
0.26723704312218843


# Setting up the multivariate ConvGRU

In [9]:
print(f"{X_train.shape=}")
print(f"{Y_train.shape=}")
print(f"{X_valid.shape=}")
print(f"{Y_valid.shape=}")

print(f"{X_train.shape=}")
print(f"{Y_train[:,2::3].shape=}")
print(f"{X_valid.shape=}")
print(f"{Y_valid[:,2::3].shape=}")

X_train.shape=(1501, 168, 3)
Y_train.shape=(1501, 168, 24)
X_valid.shape=(429, 168, 3)
Y_valid.shape=(429, 168, 24)
X_train.shape=(1501, 168, 3)
Y_train[:,2::3].shape=(1501, 56, 24)
X_valid.shape=(429, 168, 3)
Y_valid[:,2::3].shape=(429, 56, 24)


In [10]:
gru = keras.models.Sequential([
    keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]),
    keras.layers.GRU(multi_neurons_first, return_sequences=True),
    keras.layers.GRU(multi_neurons_second, return_sequences=True),
    keras.layers.Dense(horizon)
])

gru.compile(loss=loss, optimizer=optimizer)
gru.fit(X_train, Y_train[:,1::1], epochs=epochs, batch_size=100, validation_data=(X_valid, Y_valid[:,1::1]))
Y_pred_gru = gru.predict(X_test)

print(MSE(Y_test[:,-1], Y_pred_gru[:,-1]))

Epoch 1/2


ValueError: in user code:

    File "C:\Users\andre\anaconda3\envs\fysstk_tf\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\andre\anaconda3\envs\fysstk_tf\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\andre\anaconda3\envs\fysstk_tf\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\andre\anaconda3\envs\fysstk_tf\lib\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "C:\Users\andre\anaconda3\envs\fysstk_tf\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\andre\anaconda3\envs\fysstk_tf\lib\site-packages\keras\losses.py", line 141, in __call__
